### Постановка задачи
Загрузить данные по энергопотреблению всех зданий в оперативную память и добиться ее минимального расхода

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

In [2]:
for type_ in ["f2", "f4"]:
    print (np.finfo(type_))
for type_ in ["i1", "i2", "i4"]:
    print (np.iinfo(type_))

Machine parameters for float16
---------------------------------------------------------------
precision =   3   resolution = 1.00040e-03
machep =    -10   eps =        9.76562e-04
negep =     -11   epsneg =     4.88281e-04
minexp =    -14   tiny =       6.10352e-05
maxexp =     16   max =        6.55040e+04
nexp =        5   min =        -max
---------------------------------------------------------------

Machine parameters for float32
---------------------------------------------------------------
precision =   6   resolution = 1.0000000e-06
machep =    -23   eps =        1.1920929e-07
negep =     -24   epsneg =     5.9604645e-08
minexp =   -126   tiny =       1.1754944e-38
maxexp =    128   max =        3.4028235e+38
nexp =        8   min =        -max
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
-------------------------------------------------------

### Загрузка данных

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

### Потребление памяти

In [4]:
print ("Строения: ", buildings.memory_usage().sum() / 1024**2, "Мб")
print ("Погода: ", weather.memory_usage().sum() / 1024**2, "Мб")
print ("Энергопотребление: ", energy.memory_usage().sum() / 1024**2, "Мб")

Строения:  0.0664520263671875 Мб
Погода:  9.597572326660156 Мб
Энергопотребление:  368.06988525390625 Мб


### Функция оптимизация памяти

In [5]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на',
          round(start_mem - end_mem, 2),
          'Мб (минус',
          round(100 * (start_mem - end_mem) / start_mem, 1),
         '%)')
    return df

### Оптимизация памяти: строения

In [6]:
buildings = reduce_mem_usage(buildings)
print (buildings.info())

Потребление памяти меньше на 0.05 Мб (минус 73.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   site_id      1449 non-null   int8    
 1   building_id  1449 non-null   int16   
 2   primary_use  1449 non-null   category
 3   square_feet  1449 non-null   int32   
 4   year_built   675 non-null    float16 
 5   floor_count  355 non-null    float16 
dtypes: category(1), float16(2), int16(1), int32(1), int8(1)
memory usage: 17.8 KB
None


### Отимизация: погода

In [7]:
weather = reduce_mem_usage(weather)
print (weather.info())

Потребление памяти меньше на 6.53 Мб (минус 68.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   site_id             139773 non-null  int8          
 1   timestamp           139773 non-null  datetime64[ns]
 2   air_temperature     139718 non-null  float16       
 3   cloud_coverage      70600 non-null   float16       
 4   dew_temperature     139660 non-null  float16       
 5   precip_depth_1_hr   89484 non-null   float16       
 6   sea_level_pressure  129155 non-null  float16       
 7   wind_direction      133505 non-null  float16       
 8   wind_speed          139469 non-null  float16       
dtypes: datetime64[ns](1), float16(7), int8(1)
memory usage: 3.1 MB
None


### Оптимизация: энергопотребление

In [8]:
energy = reduce_mem_usage(energy)
print (energy.info())

Потребление памяти меньше на 195.54 Мб (минус 53.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060910 entries, 0 to 12060909
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   building_id    int16         
 1   meter          int8          
 2   timestamp      datetime64[ns]
 3   meter_reading  float32       
dtypes: datetime64[ns](1), float32(1), int16(1), int8(1)
memory usage: 172.5 MB
None


### Объединение данных

In [9]:
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy = pd.merge(left=energy.set_index(["timestamp", "site_id"]),
                  right=weather.set_index(["timestamp", "site_id"]),
                  how="left", left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["site_id", "meter"], axis=1)
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060910 entries, 0 to 12060909
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   timestamp           datetime64[ns]
 1   building_id         int16         
 2   meter_reading       float32       
 3   primary_use         category      
 4   square_feet         int32         
 5   year_built          float16       
 6   floor_count         float16       
 7   air_temperature     float16       
 8   cloud_coverage      float16       
 9   dew_temperature     float16       
 10  precip_depth_1_hr   float16       
 11  sea_level_pressure  float16       
 12  wind_direction      float16       
 13  wind_speed          float16       
dtypes: category(1), datetime64[ns](1), float16(9), float32(1), int16(1), int32(1)
memory usage: 425.6 MB
None


### Исследование диапазона данных

In [10]:
print ("wind speed:", sorted(energy["wind_speed"].unique()))
print ("cloud coverage:", sorted(energy["cloud_coverage"].unique()))
print ("precip depth:", sorted(energy["precip_depth_1_hr"].unique()))

wind speed: [0.0, 0.5, 1.0, 1.3, 2.1, 2.2, 2.6, 3.0, 3.1, 3.6, 4.1, nan, 1.5, 1.6, 2.0, 3.2, 4.0, 4.6, 5.0, 5.1, 5.7, 6.0, 6.1, 6.2, 6.7, 7.0, 7.2, 7.7, 8.0, 8.2, 8.7, 8.8, 9.0, 9.3, 9.8, 10.0, 10.3, 10.8, 11.0, 11.3, 11.8, 12.0, 12.4, 12.8, 12.9, 13.0, 13.4, 13.9, 14.0, 14.4, 14.9, 15.0, 15.4, 16.0, 16.5, 17.0, 18.0, 18.5, 19.0]
cloud coverage: [6.0, nan, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 7.0, 8.0, 9.0]
precip depth: [nan, -1.0, 0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 45.0, 46.0, 47.0, 48.0, 50.0, 51.0, 53.0, 55.0, 56.0, 58.0, 60.0, 61.0, 64.0, 66.0, 68.0, 69.0, 70.0, 71.0, 73.0, 74.0, 76.0, 78.0, 79.0, 81.0, 83.0, 84.0, 86.0, 89.0, 91.0, 94.0, 97.0, 98.0, 99.0, 102.0, 103.0, 104.0, 105.0, 107.0, 109.0, 112.0, 113.0, 114.0, 119.0, 122.0, 124.0, 127.0, 130.0, 132.0, 135.0, 137.0, 140.

### Приведение к целым типам

In [11]:
def round_fillna (df, columns):
    for col in columns:
        type_ = "int8"
        if col in ["wind_direction", "year_built", "precip_depth_1_hr"]:
            type_ = "int16"
        if col == "precip_depth_1_hr":
            df[col] = df[col].apply(lambda x:0 if x<0 else x)
        df[col] = np.round(df[col].fillna(value=0)).astype(type_)
    return df

In [12]:
energy = round_fillna(energy, ["wind_direction", "wind_speed",
        "cloud_coverage", "precip_depth_1_hr",
        "year_built", "floor_count"])
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060910 entries, 0 to 12060909
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   timestamp           datetime64[ns]
 1   building_id         int16         
 2   meter_reading       float32       
 3   primary_use         category      
 4   square_feet         int32         
 5   year_built          int16         
 6   floor_count         int8          
 7   air_temperature     float16       
 8   cloud_coverage      int8          
 9   dew_temperature     float16       
 10  precip_depth_1_hr   int16         
 11  sea_level_pressure  float16       
 12  wind_direction      int16         
 13  wind_speed          int8          
dtypes: category(1), datetime64[ns](1), float16(3), float32(1), int16(4), int32(1), int8(3)
memory usage: 391.1 MB
None


### Удаление отработанных данных



In [14]:
del buildings
del weather
del energy